In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, exp
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME



import numpy as np


#from data_preparation import *

In [2]:
# Load the data
df = pd.read_csv("lpmc01.dat", sep = '\t')
df['age_scaled'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']
#df['cost_transit_scenario_2'] = df['cost_transit']*0.8

database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')


# Define new variables:
# # Define driving cost 
# df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
# cost_driving = Variable(cost_driving_ccharge + cost_driving_fuel)

# # Define time taken by each mode of transport
# df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']
# dur_pt = Variable(dur_pt_access + dur_pt_rail + dur_pt_int + dur_pt_bus)


# Define transport availability
# Assume pt, walking, cycle always available, with car availability depending on number of cars per household. From the data, 
# people without driving licenses choose driving as their mode of transport (eg. row 28). 
av_drive =  (car_ownership > 0)
av_pt =1
av_walk = 1
av_cycle = 1

variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


In [3]:
# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

# Model 0

In [4]:
all_results = {}

In [5]:
print("Columns in the database:")
print(database.data.columns)

Columns in the database:
Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent', 'age_scaled',
       'cost_driving', 'dur_pt'],
      dtype='object')


In [6]:
# Load the data 

# Define ASCs
ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)
B_TIME = Beta('B_TIME', 0, None, None, 0)

# Define utility functions for each alternative
V_WALK =  B_TIME * dur_walking
V_CYCLE = ASC_CYCLE + B_TIME * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + B_TIME * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * cost_driving + B_TIME * dur_driving

# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

# Specify the model using a log-logit function
model_0 = loglogit(V, availability, travel_mode)

# Create Biogeme object
biogeme = bio.BIOGEME(database, model_0)
biogeme.modelName = "Model_0"

# Estimate parameters
results = biogeme.estimate()

# Display estimation results
print("Estimation results for Model 0:")
print(results.get_estimated_parameters())

# Optional: To calculate number of rows with car ownership 0 but driving mode chosen
driving_without_car = df[(df['car_ownership'] == 0) & (df['travel_mode'] == 4)]
num_rows_driving_without_car = driving_without_car.shape[0]
#print("Number of rows where car ownership is 0 but travel mode is driving:", num_rows_driving_without_car)
all_results['Model_0'] = results

Estimation results for Model 0:
              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE -3.778171      0.103171   -36.620582           0.0
ASC_DRIVE -1.288001      0.079431   -16.215312           0.0
ASC_PT    -0.525659      0.054777    -9.596424           0.0
B_COST    -0.190833      0.014634   -13.039926           0.0
B_TIME    -5.598762      0.203407   -27.524936           0.0


In [7]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

# Update utility functions with alternative-specific time parameters
V_WALK = B_TIME_WALK * dur_walking
V_CYCLE = ASC_CYCLE + B_TIME_CYCLE * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + B_TIME_PT * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * cost_driving + B_TIME_DRIVE * dur_driving

# Redefine the model
model_1 = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

# Create Biogeme object for Model 1
biogeme_model_1 = bio.BIOGEME(database, model_1)
biogeme_model_1.modelName = "Model_1"

# Estimate parameters for Model 1
results_model_1 = biogeme_model_1.estimate()
all_results['Model_1'] = results_model_1

# Display estimation results
print("Estimation results for Model 1:")
print(results_model_1.get_estimated_parameters())

Estimation results for Model 1:
                 Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE    -4.602378      0.197139   -23.345860           0.0
ASC_DRIVE    -2.115318      0.144923   -14.596200           0.0
ASC_PT       -2.599549      0.146553   -17.737963           0.0
B_COST       -0.180812      0.017547   -10.304242           0.0
B_TIME_CYCLE -6.462945      0.483849   -13.357357           0.0
B_TIME_DRIVE -6.623754      0.379926   -17.434326           0.0
B_TIME_PT    -3.494668      0.244093   -14.316930           0.0
B_TIME_WALK  -9.065278      0.456118   -19.874846           0.0


In [8]:
# Retrieve log-likelihoods and number of parameters
model_0_loglike = results.data.logLike       # Log-likelihood for Model 0
model_0_numParam = len(results.get_beta_values())   # Number of parameters in Model 0

model_1_loglike = results_model_1.data.logLike       # Log-likelihood for Model 1
model_1_numParam = len(results_model_1.get_beta_values())   # Number of parameters in Model 1

# Perform the likelihood ratio test
alpha = 0.05
lr_test_result = likelihood_ratio_test(
    [model_0_loglike, model_0_numParam],
    [model_1_loglike, model_1_numParam],
    alpha # Significance level of 0.01
)

# Display the results (accessing values by index)
print("Likelihood Ratio Test Results:")
print(f" {lr_test_result[0]}")           # LR Statistic


Likelihood Ratio Test Results:
 H0 can be rejected at level 5.0%


Model_pref = model_1

# Model 2

In [9]:
# Define interaction terms between ASCs and age_scaled
ASC_CYCLE_AGE = ASC_CYCLE + Beta('ASC_CYCLE_AGE', 0, None, None, 0) * age_scaled
ASC_PT_AGE = ASC_PT + Beta('ASC_PT_AGE', 0, None, None, 0) * age_scaled
ASC_DRIVE_AGE = ASC_DRIVE + Beta('ASC_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Utility functions with interaction terms
V_WALK = B_TIME_WALK * dur_walking
V_CYCLE = ASC_CYCLE_AGE + B_TIME_CYCLE * dur_cycling
V_PT = ASC_PT_AGE + B_COST * cost_transit + B_TIME_PT * dur_pt
V_DRIVE = ASC_DRIVE_AGE + B_COST * cost_driving + B_TIME_DRIVE * dur_driving

# Specify the model
model_2_spec1 = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

# Estimate Model 2 - Specification 1
biogeme_spec1 = bio.BIOGEME(database, model_2_spec1)
biogeme_spec1.modelName = "Model_2_spec1"
results_spec1 = biogeme_spec1.estimate()
all_results['Model_2_spec1'] = results_spec1

In [10]:
# Define interaction terms between travel time parameters and age_scaled
ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0) * age_scaled
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) * age_scaled
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0) * age_scaled
B_TIME_DRIVE_AGE = Beta('B_TIME_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Updated utility functions with age interaction for travel time
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE) * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + (B_TIME_PT + B_TIME_PT_AGE) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * cost_driving + (B_TIME_DRIVE + B_TIME_DRIVE_AGE) * dur_driving

# V_WALK = B_TIME_WALK * dur_walking
# V_CYCLE = ASC_CYCLE + B_TIME_CYCLE * dur_cycling
# V_PT = ASC_PT + B_COST * cost_transit + B_TIME_PT * dur_pt
# V_DRIVE = ASC_DRIVE + B_COST * cost_driving + B_TIME_DRIVE * dur_driving



# Specify the model
model_2_spec2 = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

# Estimate Model 2 - Specification 2
biogeme_spec2 = bio.BIOGEME(database, model_2_spec2)
biogeme_spec2.modelName = "Model_2_spec2_"
results_spec2_ = biogeme_spec2.estimate()
all_results['Model_2_spec2_'] = results_spec2_

In [11]:
print("Estimation results for Model 2 - Specification 1 (Interaction with ASCs):")
print(results_spec1.get_estimated_parameters())

# Display estimation results for Model 2 - Specification 2
print("\nEstimation results for Model 2 - Specification 2 (Interaction with Travel Time):")
print(results_spec2_.get_estimated_parameters())

Estimation results for Model 2 - Specification 1 (Interaction with ASCs):
                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE     -4.611336      0.199462   -23.118840  0.000000e+00
ASC_CYCLE_AGE  0.095215      0.077758     1.224505  2.207617e-01
ASC_DRIVE     -2.142379      0.147746   -14.500429  0.000000e+00
ASC_DRIVE_AGE  0.296494      0.045540     6.510669  7.481704e-11
ASC_PT        -2.610719      0.148465   -17.584769  0.000000e+00
ASC_PT_AGE     0.189040      0.049284     3.835696  1.252090e-04
B_COST        -0.182406      0.017625   -10.349493  0.000000e+00
B_TIME_CYCLE  -6.479459      0.485375   -13.349399  0.000000e+00
B_TIME_DRIVE  -6.554869      0.378414   -17.321956  0.000000e+00
B_TIME_PT     -3.478239      0.242092   -14.367451  0.000000e+00
B_TIME_WALK   -9.119824      0.464604   -19.629222  0.000000e+00

Estimation results for Model 2 - Specification 2 (Interaction with Travel Time):
                     Value  Rob. Std err  Rob. t-test  Rob. p-va

In [12]:
# Retrieve log-likelihoods and number of parameters for Model_pref and Model 2 specifications
model_1_loglike = results.data.logLike      
model_1_numParam = len(results.get_beta_values())  

spec1_loglike = results_spec1.data.logLike       
spec1_numParam = len(results_spec1.get_beta_values())   

spec2_loglike = results_spec2_.data.logLike       
spec2_numParam = len(results_spec2_.get_beta_values())   

# Perform the likelihood ratio test for Model 1 vs Model 2 - Specification 1
alpha = 0.00001
lr_test_spec1 = likelihood_ratio_test(
    [model_1_loglike, model_1_numParam],
    [spec1_loglike, spec1_numParam],
    alpha  # Significance level of alpha
)

# Display results for Model 0 vs Model 2 - Specification 1
print("Likelihood Ratio Test Results for Model 0 vs Model 2 - Specification 1:")
print(f" {lr_test_spec1[0]}")         

# Perform the likelihood ratio test for Model 0 vs Model 2 - Specification 2
lr_test_spec2 = likelihood_ratio_test(
    [model_0_loglike, model_0_numParam],
    [spec2_loglike, spec2_numParam],
    alpha 
)

# Display results for Model 0 vs Model 2 - Specification 2
print("\nLikelihood Ratio Test Results for Model 0 vs Model 2 - Specification 2:")
print(f": {lr_test_spec2[0]}")          




Likelihood Ratio Test Results for Model 0 vs Model 2 - Specification 1:
 H0 can be rejected at level 0.0%

Likelihood Ratio Test Results for Model 0 vs Model 2 - Specification 2:
: H0 can be rejected at level 0.0%


In [13]:
print(model_1_numParam)
print(spec2_numParam)

5
12


In [14]:

comparison_table, _ = compile_estimation_results(all_results)
display(comparison_table)

,Model_0,Model_1,Model_2_spec1,Model_2_spec2_
Number of estimated parameters,5,8,11,12
Sample size,5000,5000,5000,5000
Final log likelihood,-4581.895651,-4223.985956,-4202.066677,-4193.749471
Akaike Information Criterion,9173.791303,8463.971913,8426.133354,8411.498941
Bayesian Information Criterion,9206.377269,8516.109458,8497.822479,8489.70526
ASC_CYCLE (t-test),-3.78 (-36.6),-4.6 (-23.3),-4.61 (-23.1),-4.62 (-23)
ASC_DRIVE (t-test),-1.29 (-16.2),-2.12 (-14.6),-2.14 (-14.5),-2.16 (-14.4)
ASC_PT (t-test),-0.526 (-9.6),-2.6 (-17.7),-2.61 (-17.6),-2.61 (-17.4)
B_COST (t-test),-0.191 (-13),-0.181 (-10.3),-0.182 (-10.3),-0.183 (-10.3)
B_TIME (t-test),-5.6 (-27.5),,,


model_pref = model 2 (spec 1)

# Model 3
(Box_Cox)

In [15]:
variable_name = 'dur_pt'  # Replace with your variable name
if variable_name in database.data.columns:
    print(f"'{variable_name}' exists in the database.")
else:
    print(f"'{variable_name}' does NOT exist in the database.")

'dur_pt' exists in the database.


In [16]:
# Box-Cox Transformation for costs
lambda_cost = Beta('lambda_cost', 1, -10, 10, 0)
boxcox_cost_transit = boxcox(cost_transit, lambda_cost)
boxcox_cost_driving = boxcox(cost_driving, lambda_cost)

# Define interaction terms between ASCs and age_scaled
ASC_CYCLE_AGE = ASC_CYCLE + Beta('ASC_CYCLE_AGE', 0, None, None, 0) * age_scaled
ASC_PT_AGE = ASC_PT + Beta('ASC_PT_AGE', 0, None, None, 0) * age_scaled
ASC_DRIVE_AGE = ASC_DRIVE + Beta('ASC_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Utility functions with Box-Cox transformation
V_WALK = B_TIME_WALK * dur_walking
V_CYCLE = ASC_CYCLE_AGE + B_TIME_CYCLE * dur_cycling
V_PT = ASC_PT_AGE + B_COST * boxcox_cost_transit + B_TIME_PT * dur_pt
V_DRIVE = ASC_DRIVE_AGE + B_COST * boxcox_cost_driving + B_TIME_DRIVE * dur_driving

# Specify the model with Box-Cox transformation
model_3_boxcox = loglogit({1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}, availability, travel_mode)

# Create Biogeme object
biogeme_model_3 = bio.BIOGEME(database, model_3_boxcox)
biogeme_model_3.modelName = "Model_3_BoxCox"

# Estimate Model 3
results_model_3 = biogeme_model_3.estimate()

# Log-likelihood and number of parameters for Model 3
model_3_boxcox_loglike = results_model_3.data.logLike
model_3_boxcox_numParam = results_model_3.get_estimated_parameters().shape[0]

# Store results for comparison
all_results['Model_3_BoxCox'] = results_model_3

# Display results
print("Estimation results for Model 3 (Box-Cox Transformation):")
print(results_model_3.get_estimated_parameters())

Estimation results for Model 3 (Box-Cox Transformation):
                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE     -4.735370      0.214838   -22.041553  0.000000e+00
ASC_CYCLE_AGE  0.092800      0.077886     1.191491  2.334608e-01
ASC_DRIVE     -2.853291      0.207382   -13.758641  0.000000e+00
ASC_DRIVE_AGE  0.293056      0.045106     6.497073  8.189760e-11
ASC_PT        -2.784977      0.158434   -17.578200  0.000000e+00
ASC_PT_AGE     0.194193      0.050281     3.862127  1.124042e-04
B_COST        -0.460248      0.052159    -8.823889  0.000000e+00
B_TIME_CYCLE  -6.980558      0.514153   -13.576817  0.000000e+00
B_TIME_DRIVE  -6.213037      0.377630   -16.452731  0.000000e+00
B_TIME_PT     -3.762030      0.247651   -15.190825  0.000000e+00
B_TIME_WALK   -9.540978      0.506012   -18.855238  0.000000e+00
lambda_cost    0.323788      0.081119     3.991540  6.564567e-05


In [17]:
# Retrieve log-likelihoods and number of parameters for Model_pref and Model 2 specifications

spec1_loglike = results_spec1.data.logLike       
spec1_numParam = len(results_spec1.get_beta_values())   

model_3_boxcox_loglike = results_model_3.data.logLike       
model_3_boxcox_numParam = len(results_model_3.get_beta_values())   

# Perform the likelihood ratio test for Model 1 vs Model 2 - Specification 1
alpha = 0.01
lr_test_spec1 = likelihood_ratio_test(
    [spec1_loglike, spec1_numParam],
    [model_3_boxcox_loglike, model_0_numParam],
    alpha  # Significance level of alpha
)

# Display results for Model 0 vs Model 2 - Specification 1
print("Likelihood Ratio Test Results for Model 3 vs Model 2 - Specification 1:")
print(f" {lr_test_spec1[0]}")         
      


Likelihood Ratio Test Results for Model 3 vs Model 2 - Specification 1:
 H0 cannot be rejected at level 1.0%


In [18]:
comparison_table, _ = compile_estimation_results(all_results)
display(comparison_table)



,Model_0,Model_1,Model_2_spec1,Model_2_spec2_,Model_3_BoxCox
Number of estimated parameters,5,8,11,12,12
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4581.895651,-4223.985956,-4202.066677,-4193.749471,-4203.35605
Akaike Information Criterion,9173.791303,8463.971913,8426.133354,8411.498941,8430.7121
Bayesian Information Criterion,9206.377269,8516.109458,8497.822479,8489.70526,8508.918418
ASC_CYCLE (t-test),-3.78 (-36.6),-4.6 (-23.3),-4.61 (-23.1),-4.62 (-23),-4.74 (-22)
ASC_DRIVE (t-test),-1.29 (-16.2),-2.12 (-14.6),-2.14 (-14.5),-2.16 (-14.4),-2.85 (-13.8)
ASC_PT (t-test),-0.526 (-9.6),-2.6 (-17.7),-2.61 (-17.6),-2.61 (-17.4),-2.78 (-17.6)
B_COST (t-test),-0.191 (-13),-0.181 (-10.3),-0.182 (-10.3),-0.183 (-10.3),-0.46 (-8.82)
B_TIME (t-test),-5.6 (-27.5),,,,


modelpref = model2, spec_1

In [19]:
print(database.data.columns)

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent', 'age_scaled',
       'cost_driving', 'dur_pt'],
      dtype='object')


# Model 4

In [20]:
# Define ASC interaction terms with age_scaled
ASC_CYCLE_AGE = ASC_CYCLE + Beta('ASC_CYCLE_AGE', 0, None, None, 0) * age_scaled
ASC_PT_AGE = ASC_PT + Beta('ASC_PT_AGE', 0, None, None, 0) * age_scaled
ASC_DRIVE_AGE = ASC_DRIVE + Beta('ASC_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Define utility functions
V_WALK = B_TIME_WALK * dur_walking
V_CYCLE = ASC_CYCLE_AGE + B_TIME_CYCLE * dur_cycling
V_PT = ASC_PT_AGE + B_COST * cost_transit + B_TIME_PT * dur_pt
V_DRIVE = ASC_DRIVE_AGE + B_COST * cost_driving + B_TIME_DRIVE * dur_driving
# Define utility dictionary
V = {1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}

mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)
nest_a = OneNestForNestedLogit(nest_param=mu_a, list_of_alternatives=[1, 2], name='slow modes')
nest_b = OneNestForNestedLogit(nest_param=mu_b, list_of_alternatives=[3, 4], name='faster modes')
nests = NestsForNestedLogit(choice_set=list(V), tuple_of_nests=(nest_a, nest_b))




# Lognested model
logprob_m4 = lognested(V, None, nests, travel_mode)

# Create Biogeme object
biogeme_model_4 = bio.BIOGEME(database, logprob_m4)
biogeme_model_4.modelName = "Model_4_Nested"

# Estimate Model 4
results_model_4 = biogeme_model_4.estimate()

# Log-likelihood and number of parameters for Model 4
model_4_loglike = results_model_4.data.logLike
model_4_numParam = results_model_4.get_estimated_parameters().shape[0]

# Store results for comparison
all_results['Model_4'] = results_model_4

# Display results
print("Estimation results for Model 4 (Nested Logit):")
print(results_model_4.get_estimated_parameters())

Estimation results for Model 4 (Nested Logit):
                   Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CYCLE      -6.384210      0.301869   -21.148923  0.000000e+00
ASC_CYCLE_AGE   0.151616      0.100386     1.510333  1.309586e-01
ASC_DRIVE      -2.589141      0.172892   -14.975442  0.000000e+00
ASC_DRIVE_AGE   0.334196      0.052330     6.386350  1.698923e-10
ASC_PT         -3.253413      0.222462   -14.624602  0.000000e+00
ASC_PT_AGE      0.178927      0.059520     3.006180  2.645521e-03
B_COST         -0.251141      0.035253    -7.123915  1.048939e-12
B_TIME_CYCLE   -6.112983      0.792062    -7.717811  1.176836e-14
B_TIME_DRIVE   -9.270750      1.012808    -9.153516  0.000000e+00
B_TIME_PT      -4.907944      0.560997    -8.748614  0.000000e+00
B_TIME_WALK   -11.214684      0.610788   -18.361013  0.000000e+00
mu_a            0.591774      0.046586    12.702886  0.000000e+00
mu_b            0.700262      0.083919     8.344511  0.000000e+00


In [21]:
# Retrieve log-likelihoods and number of parameters for Model_pref and Model 2 specifications

spec1_loglike = results_spec1.data.logLike       
spec1_numParam = len(results_spec1.get_beta_values())   

model_4_loglike = results_model_4.data.logLike       
model_4_numParam = len(results_model_4.get_beta_values())   

# Perform the likelihood ratio test for Model 1 vs Model 2 - Specification 1
alpha = 0.01
lr_test_spec1 = likelihood_ratio_test(
    [spec1_loglike, spec1_numParam],
    [model_4_loglike, model_4_numParam],
    alpha  # Significance level of alpha
)

# Display results for Model 0 vs Model 2 - Specification 1
print("Likelihood Ratio Test Results for Model 4 vs Model 2 - Specification 1:")
print(f" {lr_test_spec1[0]}")      

Likelihood Ratio Test Results for Model 4 vs Model 2 - Specification 1:
 H0 can be rejected at level 1.0%


In [22]:
comparison_table, _ = compile_estimation_results(all_results)
display(comparison_table)

,Model_0,Model_1,Model_2_spec1,Model_2_spec2_,Model_3_BoxCox,Model_4
Number of estimated parameters,5,8,11,12,12,13
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4581.895651,-4223.985956,-4202.066677,-4193.749471,-4203.35605,-4180.831789
Akaike Information Criterion,9173.791303,8463.971913,8426.133354,8411.498941,8430.7121,8387.663578
Bayesian Information Criterion,9206.377269,8516.109458,8497.822479,8489.70526,8508.918418,8472.38709
ASC_CYCLE (t-test),-3.78 (-36.6),-4.6 (-23.3),-4.61 (-23.1),-4.62 (-23),-4.74 (-22),-6.38 (-21.1)
ASC_DRIVE (t-test),-1.29 (-16.2),-2.12 (-14.6),-2.14 (-14.5),-2.16 (-14.4),-2.85 (-13.8),-2.59 (-15)
ASC_PT (t-test),-0.526 (-9.6),-2.6 (-17.7),-2.61 (-17.6),-2.61 (-17.4),-2.78 (-17.6),-3.25 (-14.6)
B_COST (t-test),-0.191 (-13),-0.181 (-10.3),-0.182 (-10.3),-0.183 (-10.3),-0.46 (-8.82),-0.251 (-7.12)
B_TIME (t-test),-5.6 (-27.5),,,,,


model_pref = model 4

# Market Shares

The weight $w_g$ associated with segment $g$ is defined as
$$
w_g = \frac{N_g}{N}\frac{S}{S_g}.
$$

In [23]:
populations = {
    'male_44_less':  2926408,
    'male_45_more': 1379198 ,
    'female_44_less':  2841379,
    'female_45_more':  1519948,
    }

total_pop = sum(populations.values())
total_pop

8666933

In [24]:
filters = {
    'male_45_more': (df.age >= 45) & (df.female == 0),
    'male_44_less': (df.age < 45) & (df.female == 0),
    'female_45_more': (df.age >= 45) & (df.female == 1),
    'female_44_less': (df.age < 45) & (df.female == 1),
}


In [25]:
sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(sample_segments)

total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

weights = {
    segment_name: populations[segment_name] * total_sample / (segment_size * total_pop)
    for segment_name, segment_size in sample_segments.items()
}
print(weights)

{'male_45_more': np.int64(896), 'male_44_less': np.int64(1442), 'female_45_more': np.int64(984), 'female_44_less': np.int64(1678)}
Sample size: 5000
{'male_45_more': np.float64(0.8880208732101985), 'male_44_less': np.float64(1.1707769945566922), 'female_45_more': np.float64(0.8911241160085213), 'female_44_less': np.float64(0.9768812522649147)}


In [26]:
total_sample = sum(sample_segments.values())
total_sample

np.int64(5000)

In [27]:
from biogeme import models

for segment_name, segment_rows in filters.items():
    df.loc[segment_rows, 'weight'] = weights[segment_name]


prob_walk = models.nested(V, None, nests, 1)
prob_cycle = models.nested(V, None, nests, 2)
prob_pt = models.nested(V, None, nests, 3)
prob_car = models.nested(V, None, nests, 4)


weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
}

database = db.Database('london', df)


In [28]:

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)
simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.888021,0.298544,0.687323,3.604518e-03,0.010528
1,0.976881,0.139332,0.688139,1.236051e-01,0.048924
2,0.891124,0.227017,0.757269,2.614680e-05,0.015688
3,0.888021,0.344664,0.629458,5.600291e-03,0.020278
4,0.891124,0.177937,0.366137,4.170913e-01,0.038835
...,...,...,...,...,...
4995,0.888021,0.341517,0.635327,6.973523e-03,0.016182
4996,1.170777,0.434123,0.504826,2.005082e-02,0.040999
4997,0.976881,0.236921,0.385207,3.179543e-01,0.059917
4998,0.976881,0.278906,0.712042,1.111337e-04,0.008940


In [29]:

market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

biogeme_model_4.bootstrap_samples = 100
results_bootstrapping = biogeme_model_4.estimate(run_bootstrap=True)

betas = biogeme_model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)

display(left)

display(right)

Market share for pt: 35.6%
Market share for car: 43.6%
Market share for walk: 17.4%
Market share for cycling: 3.4%


100%|██████████| 100/100 [01:37<00:00,  1.03it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.888021,0.262350,0.660012,2.158728e-03,0.007623
1,0.976881,0.126064,0.662791,1.029152e-01,0.041158
2,0.891124,0.201766,0.735350,7.838843e-06,0.011979
3,0.888021,0.315711,0.608337,3.432307e-03,0.015977
4,0.891124,0.158720,0.344333,3.914274e-01,0.030476
...,...,...,...,...,...
4995,0.888021,0.322323,0.619332,4.562063e-03,0.012998
4996,1.170777,0.416665,0.483536,1.386825e-02,0.033220
4997,0.976881,0.224290,0.364714,2.943650e-01,0.050234
4998,0.976881,0.258222,0.696420,4.210524e-05,0.006267


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.888021,0.326154,0.724564,5.600323e-03,0.014831
1,0.976881,0.151704,0.717677,1.428742e-01,0.055223
2,0.891124,0.247712,0.780583,7.752043e-05,0.018776
3,0.888021,0.365757,0.659312,8.247155e-03,0.025732
4,0.891124,0.193188,0.393047,4.483553e-01,0.046605
...,...,...,...,...,...
4995,0.888021,0.357239,0.656652,9.972395e-03,0.020262
4996,1.170777,0.455981,0.524216,2.709699e-02,0.049016
4997,0.976881,0.248387,0.403250,3.390928e-01,0.067226
4998,0.976881,0.296361,0.733524,2.472487e-04,0.011558


In [30]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")
print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")



Market share for pt: 35.6% CI: [33.5%-37.8%]
Market share for car: 43.6% CI: [41.3%-46.0%]
Market share for walk: 17.4% CI: [16.1%-18.7%]
Market share for cycling: 3.4% CI: [2.7%-4.1%]


In [31]:
# Example mapping
labels = {1: 'walk', 2: 'cycling', 3: 'pt', 4: 'car'}

# Map the travel_mode column to the labels
df['mode_label'] = df['travel_mode'].map(labels)

# Calculate market shares
market_shares = (
    df['mode_label']
    .value_counts(normalize=True)  # Get proportions
    .sort_index()  # Ensure consistent order
    * 100  # Convert to percentage
)

# Print market shares
for mode, share in market_shares.items():
    print(f"Market share for {mode}: {share:.1f}%")

Market share for car: 44.0%
Market share for cycling: 3.3%
Market share for pt: 35.3%
Market share for walk: 17.4%


# Forecasting

Scenario 2: decrease rail cost by 20%

In [38]:
data_filtered = df

In [39]:
for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]


prob_walk = models.nested(V, None, nests, 1)
prob_cycle = models.nested(V, None, nests, 2)
prob_pt = models.nested(V, None, nests, 3)
prob_car = models.nested(V, None, nests, 4)


weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
}

data_filtered["cost_driving_fuel"] += 1.5
database = db.Database('london', data_filtered)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_forecasting.get_beta_values())
display(simulated_values)


Column mode_label in the database does contain object


BiogemeError: Column mode_label in the database does contain object

In [35]:
data_filtered = df
# new_cost_transit = cost_transit*0.8
# V_PT_scenario_2 = ASC_PT_AGE + B_COST * new_cost_transit + B_TIME_PT * dur_pt

# prob_walk_scenario_2 = models.nested(V, None, nests, 1)
# prob_cycle_scenario_2 = models.nested(V, None, nests, 2)
# prob_pt_scenario_2 = models.nested(V, None, nests, 3)
# prob_car_scenario_2 = models.nested(V, None, nests, 4)


In [37]:
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
}

data_filtered["cost_transit"] = data_filtered["cost_transit"]*0.8
database = db.Database('lpmc01_2', data_filtered)

biosim = BIOGEME(database, simulate)
simulated_values_scenario_2 = biosim.simulate(results_model_4.get_beta_values())
display(simulated_values)


Column mode_label in the database does contain object


BiogemeError: Column mode_label in the database does contain object

In [34]:
print(data_filtered['mode_label'].dtype)
print(data_filtered['mode_label'].unique())

object
['car' 'pt' 'walk' 'cycling']


In [65]:
simulated_values_scenario_2['Weighted pt'] = (
    simulated_values_scenario_2['weight'] * simulated_values_scenario_2['Prob. pt']
)
simulated_values_scenario_2['Weighted car'] = (
    simulated_values_scenario_2['weight'] * simulated_values_scenario_2['Prob. car']
)

simulated_values_scenario_2['Weighted walk'] = (
    simulated_values_scenario_2['weight'] * simulated_values_scenario_2['Prob. walk']
)
simulated_values_scenario_2['Weighted cycle'] = (
    simulated_values_scenario_2['weight'] * simulated_values_scenario_2['Prob. cycle']
)

In [66]:
market_share_pt_2 = simulated_values_scenario_2['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt_2:.1f}%')

market_share_car_2 = simulated_values_scenario_2['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car_2:.1f}%')

market_share_walk_2 = simulated_values_scenario_2['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk_2:.1f}%')

market_share_cycle_2 = simulated_values_scenario_2['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle_2:.1f}%')

biogeme_model_4.bootstrap_samples = 100
results_bootstrapping = biogeme_model_4.estimate(run_bootstrap=True)

betas_scenario_2 = biogeme_model_4.free_beta_names
b_scenario_2 = results_bootstrapping.get_betas_for_sensitivity_analysis(betas_scenario_2)
left_scenario_2, right_scenario_2 = biosim.confidence_intervals(b_scenario_2, 0.9)

display(left)

display(right)

Market share for pt: 35.6%
Market share for car: 43.6%
Market share for walk: 17.4%
Market share for cycling: 3.4%


100%|██████████| 100/100 [01:41<00:00,  1.01s/it]


KeyboardInterrupt: 

In [ ]:
left_scenario_2['Weighted pt'] = left_scenario_2['weight'] * left_scenario_2['Prob. pt']
left_scen['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")
print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

